In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline

In [32]:
total_rawdata = pd.DataFrame()
for i in range(7):
    raw_grav = pd.read_csv("20170252/" + str(i) +"/gravity.csv", header=None )
    raw_grav.pop(raw_grav.columns[1])
    
    raw_linear = pd.read_csv("20170252/" + str(i) +"/linear.csv", header=None )
    raw_linear.pop(raw_linear.columns[0])
    raw_linear.pop(raw_linear.columns[0])
    raw_gyro = pd.read_csv("20170252/" + str(i) +"/gyro.csv", header=None )
    raw_gyro.pop(raw_gyro.columns[0])
    raw_gyro.pop(raw_gyro.columns[0])
    
    
    merged_df = pd.concat([raw_grav, raw_linear], axis= 1)
    merged_df = pd.concat([merged_df, raw_gyro], axis= 1)

    print(merged_df.shape)
    ##have to deal with the number
    total_rawdata = pd.concat([total_rawdata,merged_df], axis = 0)
    total_rawdata=total_rawdata.dropna(axis=0)
    total_rawdata=total_rawdata.dropna(axis=1)

    total_rawdata.to_csv(str(i)+".csv", sep = ",", index = False, header=None)

   


 #window
"""
    count = 0
    while True:
        temp = list()
        if count + 200 > merged_df.shape[0]:
            break
        a = merged_df[count : count + 200].mean()
        count = count + 100

"""




(59497, 10)
(62570, 10)
(61982, 10)
(57940, 10)
(65387, 10)
(21338, 10)
(15825, 10)


'\n    count = 0\n    while True:\n        temp = list()\n        if count + 200 > merged_df.shape[0]:\n            break\n        a = merged_df[count : count + 200].mean()\n        count = count + 100\n\n'

In [33]:
total_rawdata.shape

(319115, 10)

In [34]:
"""
mean
energy
standard deviation
"""
count = 0 

while True:
    if count + 200> total_rawdata.shape[0]:
        break
    window_df = total_rawdata[count : count + 200]
    window_df.shape


KeyboardInterrupt: 